In [ ]:
# Experimentación: Búsqueda con Multiprocessing
import os
import time
from multiprocessing import Pool

# Configuración del experimento
DATASET_PATH = "dataset"
SEARCH_TERM = "needle"
REPEATS = 10  # Número de repeticiones por prueba


def search_in_file(file_path, search_term):
    """Realiza una búsqueda del término en un archivo dado."""
    with open(file_path, "r") as f:
        for line in f:
            if search_term in line:
                return True
    return False


def multiprocessing_search(file_paths, search_term):
    """Realiza una búsqueda paralela con multiprocessing."""
    with Pool(processes=os.cpu_count()) as pool:
        results = pool.starmap(search_in_file, [(file_path, search_term) for file_path in file_paths])
    return results


def measure_execution_time(file_paths, search_term, repeats=1):
    """Mide el tiempo de ejecución de la búsqueda con multiprocessing."""
    total_time = 0
    for _ in range(repeats):
        start_time = time.time()
        multiprocessing_search(file_paths, search_term)
        total_time += time.time() - start_time
    return total_time / repeats


def run_experiment():
    """Ejecuta el experimento con multiprocessing y registra los resultados."""
    results = []

    for folder in os.listdir(DATASET_PATH):
        folder_path = os.path.join(DATASET_PATH, folder)
        if not os.path.isdir(folder_path):
            continue
        
        file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
        size, num_files = map(int, [s.split("_")[1].replace("MB", "") for s in folder.split("_")[1::2]])
        
        mp_time = measure_execution_time(file_paths, SEARCH_TERM, REPEATS)

        results.append({
            "file_size_mb": size,
            "num_files": num_files,
            "multiprocessing_time": mp_time
        })

    return results


def main():
    """Punto de entrada principal para el experimento de multiprocessing."""
    print("Ejecutando experimentos con Multiprocessing...")
    results = run_experiment()
    print("Resultados:")
    for result in results:
        print(result)


if __name__ == "__main__":
    main()
